In [1]:
import sys
import os

#path of the project
project_path = "/home/diego-ngz/Git/thesis-tabtrans"

sys.path.append(project_path) #This helps to be able to import the data from the parent directory to other files

from utils import data, tabtrans_file, plots,attention, training, attention_file
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import pickle
import plotly.express as px
from skorch.callbacks import Checkpoint, TrainEndCheckpoint, EarlyStopping, LoadInitState, EpochScoring, Checkpoint
import skorch
import torch.nn as nn
import torch

/home/diego-ngz/Git/thesis-tabtrans


In [2]:
'''  
Steps to follow:
1. Load the data
2. separate between training and testing.
3. Separate the validation from the training set
4. Import the best hyperparameters
5. Train the model and in every epoch check the validation error, the training error, and the attention matrix
8. Stop the model when 10 epochs the entropy doesnt decrease
''' 

'  \nSteps to follow:\n1. Load the data\n2. separate between training and testing.\n3. Separate the validation from the training set\n4. Import the best hyperparameters\n5. Train the model and in every epoch check the validation error, the training error, and the attention matrix\n8. Stop the model when 10 epochs the entropy doesnt decrease\n'

In [3]:
df_id = 2
X_train, X_test, y_train, y_test, train_indices, val_indices, n_instances, n_labels, n_numerical, n_categories = data.import_data(df_id)
'''
The train_indices are the ones used for training the model
The val_indices are the ones used for validation (is the 20% of the training set)
'''
n_features = X_train.shape[1]

name_df = data.get_dataset_name(df_id)

path_of_datset = f'{project_path}/Final_models_4/{name_df}'

path_to_hyperparameters = f'{path_of_datset}/tabtrans/hyperparameter_selection'

#define the path to final_tabtrans
path_to_final_tabtrans = f'{path_of_datset}/tabtrans/final_tabtrans_cv'

sample = 100
path_of_hyper_size = f'{path_to_hyperparameters}/{sample}'
path_of_hyper_results = f'{path_of_hyper_size}/results.csv'


INFO:openml.datasets.dataset:pickle write anneal


In [4]:
#import the hyperparameters
hyperparameters = data.import_hyperparameters(path_of_hyper_results, cv = True)


n_layers = int(hyperparameters["n_layers"])
n_heads = int(hyperparameters["n_heads"])
embedding_size = int(hyperparameters["embedding_size"])
batch_size = int(hyperparameters["batch_size"])
epochs = int(hyperparameters["max_epochs_mean"])

In [5]:
#Define the model

#parameters for the model
ff_pw_size = 30  #this value because of the paper 
attn_dropout = 0.3 #paper
ff_dropout = 0.1 #paper value
aggregator = "cls"
aggregator_parameters = None
decoder_hidden_units = [128,64] #paper value [128,64]
decoder_activation_fn = nn.ReLU()
need_weights = False
numerical_passthrough = False

#module
module = training.build_module(
    n_categories, # List of number of categories
    n_numerical, # Number of numerical features
    n_heads, # Number of heads per layer
    ff_pw_size, # Size of the MLP inside each transformer encoder layer
    n_layers, # Number of transformer encoder layers    
    n_labels, # Number of output neurons
    embedding_size,
    attn_dropout, 
    ff_dropout, 
    aggregator, # The aggregator for output vectors before decoder
    rnn_aggregator_parameters=aggregator_parameters,
    decoder_hidden_units=decoder_hidden_units,
    decoder_activation_fn=decoder_activation_fn,
    need_weights=need_weights,
    numerical_passthrough=numerical_passthrough
) 



/home/diego-ngz/anaconda3/envs/tabtrans/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
from skorch.callbacks import EpochScoring
import torch
import numpy as np


# Custom EpochScoring class
class CustomEpochScoring(EpochScoring):
    def __init__(self,X, y, scoring_func, n_numerical, n_layers, n_heads, n_features, **kwargs):
        super().__init__(scoring_func, **kwargs)  # Initialize the base class with your custom scoring function
        self.X = X
        self.y = y
        self.n_numerical = n_numerical
        self.n_layers = n_layers
        self.n_heads = n_heads
        self.n_features = n_features

    def on_epoch_end(self, net, dataset_train=None, dataset_valid=None, **kwargs):
        
        # Call your custom entropy function with the extra parameters
        entropy_value = attention_file.entropy_attention_matrix(
            net, self.X, self.y, self.n_numerical, self.n_layers, self.n_heads, self.n_features
        )
        
        # You can now log or use the `entropy_value` as needed
        print(f"Entropy for this epoch: {entropy_value}")

        # Optionally, record the score in the history or process further
        # Use this for logging the entropy score into the history
        self._record_score(net.history, entropy_value)
        

In [7]:
model = skorch.NeuralNetClassifier(
    module=module,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.AdamW,
    device = "cuda", #if torch.cuda.is_available() else "cpu",
    batch_size = batch_size,
    max_epochs = epochs,
    train_split=skorch.dataset.ValidSplit(((train_indices, val_indices),)),
    callbacks=[
        CustomEpochScoring(
            scoring_func=attention_file.entropy_attention_matrix,
            name="entropy_attention",  # Give it a name
            X = X_train,
            y = y_train,
            on_train=True,  # Whether you want to compute this during training or validation
            n_numerical=10,  # Just as an example, pass the additional parameters
            n_layers=12,
            n_heads=8,
            n_features=256
        )

    ],
    optimizer__lr=1e-4,
    optimizer__weight_decay=1e-4
)

In [8]:
model = model.fit(X={
            "x_numerical": X_train[:, :n_numerical].astype(np.float32),
            "x_categorical": X_train[:, n_numerical:].astype(np.int32)
            }, 
            y=y_train.astype(np.int64)
            )

RuntimeError: The size of tensor a (22) must match the size of tensor b (26) at non-singleton dimension 1

In [ ]:
''' 
        CustomEpochScoring(
            scoring_func=attention_file.entropy_attention_matrix,
            name="entropy_attention",  # Give it a name
            X = X_train,
            y = y_train,
            on_train=True,  # Whether you want to compute this during training or validation
            n_numerical=10,  # Just as an example, pass the additional parameters
            n_layers=12,
            n_heads=8,
            n_features=256
        )
''' 